In [2]:
import pandas as pd

df = pd.read_csv('labeling_500_merged.csv')

# Krippendorff's Alpha

Krippendorff's Alpha는 여러 작업자가 동일한 데이터에 대해 얼마나 일관성 있게 레이블링했는지를 평가하는 지표로, 0에서 1 사이의 값을 가집니다. 값이 1에 가까울수록 작업자들 간에 높은 일치도가 있음을 의미하며, 0에 가까울수록 일치도가 낮다는 것을 의미합니다.

0.81 ~ 1.00: 거의 완벽한 일치 (Almost perfect agreement)

0.61 ~ 0.80: 상당한 일치 (Substantial agreement)

0.41 ~ 0.60: 중간 수준의 일치 (Moderate agreement)

0.21 ~ 0.40: 공정한 일치 (Fair agreement)

0.00 ~ 0.20: 매우 낮은 일치 (Slight agreement)


In [13]:
import pandas as pd
import numpy as np
from collections import defaultdict

def krippendorffs_alpha(df):
    data = df.to_numpy()  # DataFrame을 numpy 배열로 변환
    num_items, num_raters = data.shape
    
    # 결측값을 -1로 처리함
    data = np.where(pd.isnull(data), -1, data)
    
    # 관측된 불일치 정도 D_o 계산
    def observed_disagreement(data):
        total_disagreement = 0
        for row in data:
            value_counts = defaultdict(int)
            for val in row:
                if val != -1:  # 결측값 제외
                    value_counts[val] += 1
            for val, count in value_counts.items():
                total_disagreement += count * (num_raters - count)
        return total_disagreement / (num_raters * (num_raters - 1) * num_items)

    # 기대 불일치 정도 D_e 계산
    def expected_disagreement(data):
        total_disagreement = 0
        value_counts = defaultdict(int)
        for row in data:
            for val in row:
                if val != -1:  # 결측값 제외
                    value_counts[val] += 1
        total_ratings = sum(value_counts.values())
        for val, count in value_counts.items():
            total_disagreement += count * (total_ratings - count)
        return total_disagreement / (total_ratings * (total_ratings - 1))

    D_o = observed_disagreement(data)
    D_e = expected_disagreement(data)
    
    # 크리펜도르프의 알파 계산
    alpha = 1 - (D_o / D_e) if D_e != 0 else 1
    return alpha

In [41]:
binary_df = df[['binary_js', 'binary_ej', 'binary_km']]
ternary_df = df[['ternary_js', 'ternary_ej', 'ternary_km']]
seven_df = df[['seven_js', 'seven_ej', 'seven_km']]

print(f"이진분류 Krippendorff's Alpha: {krippendorffs_alpha(binary_df)}")
print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df)}")
print(f"에크만분류 Krippendorff's Alpha: {krippendorffs_alpha(seven_df)}")


이진분류 Krippendorff's Alpha: 0.7667751060820368
삼진분류 Krippendorff's Alpha: 0.6600503731427344
에크만분류 Krippendorff's Alpha: 0.5641765786485284


In [45]:
ternary_df = df[['ternary_js', 'ternary_ej']]
print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df)}")

삼진분류 Krippendorff's Alpha: 0.652923743976542


In [46]:
ternary_df = df[['ternary_ej', 'ternary_km']]
print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df)}")

삼진분류 Krippendorff's Alpha: 0.746399053154558


In [47]:
ternary_df = df[['ternary_km', 'ternary_js']]
print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df)}")

삼진분류 Krippendorff's Alpha: 0.5758422863586803


Krippendorff's Alpha Test

이진분류 : 0.766 -> 상당한 일치

삼진분류 : 0.660 -> 상당한 일치

에크만분류 : 0.564

# Fleiss' Kappa

In [15]:
import pandas as pd
import numpy as np

def fleiss_kappa(df):


    # 레이블을 범주형 데이터로 변환
    categories = df.apply(pd.Series.value_counts, axis=1).fillna(0)
    num_items, num_raters = df.shape
    num_categories = categories.shape[1]

    # 각 항목에 대해 일치율 계산 (P_i)
    def observed_agreement(categories):
        P_i = []
        for index, row in categories.iterrows():
            n = row.sum()  # 총 평가자 수
            P_i_value = (row ** 2).sum() - n
            P_i.append(P_i_value / (n * (n - 1)))
        return np.array(P_i)

    # 범주별로 레이블링된 비율 (p_j)
    def expected_agreement(categories):
        p_j = categories.sum(axis=0) / (num_items * num_raters)
        return p_j

    # Fleiss' Kappa 계산
    P_i = observed_agreement(categories)
    P_bar = P_i.mean()
    p_j = expected_agreement(categories)
    P_e_bar = (p_j ** 2).sum()
    
    kappa = (P_bar - P_e_bar) / (1 - P_e_bar) if 1 - P_e_bar != 0 else 1
    return kappa

In [35]:
print(f"이진분류 Fleiss' Kappa: {fleiss_kappa(binary_df)}")
print(f"삼진분류 Fleiss' Kappa: {fleiss_kappa(ternary_df)}")
print(f"에크만분류 Fleiss' Kappa: {fleiss_kappa(seven_df)}")


이진분류 Fleiss' Kappa: 0.7666195190947663
삼진분류 Fleiss' Kappa: 0.6598235888686469
에크만분류 Fleiss' Kappa: 0.5638858358724436


# LLM

In [4]:
#Use 3.12.4
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document

key_path = '/Users/jaesolshin/key/openai_key.txt'
os.environ["OPENAI_API_KEY"] = open(key_path, 'r', encoding='utf-8').read()
openai.api_key = os.environ["OPENAI_API_KEY"] 

In [5]:
import openai
import re
import time

# Sample 텍스트(50~52번 댓글)
comments = df['comment'][50:53]  # 슬라이싱 처리

# 감정 분석 함수 정의
def sentiment_by_gpt(comment):
    # OpenAI API를 사용하여 채팅 형식으로 요청
    response = openai.ChatCompletion.create(
        model="gpt-4",  # 사용할 모델
        messages=[
            {"role": "system", "content": "You are an expert sentiment analyst."},
            {"role": "user", "content": f"""문장의 감정을 "2" (긍정), "1" (중립), "0" (부정) 중 하나로 평가해 주세요. 숫자로만 응답하세요. 문장: "{comment}"\n감정:"""}
        ]
    )
    # 응답에서 메시지 내용 추출
    response_text = response['choices'][0]['message']['content'].strip()
    
    # 숫자만 추출
    score = re.search(r'\d', response_text)
    return score.group() if score else "N/A"

# 감정 분석 수행 및 결과 출력
for comment in comments:
    sentiment = sentiment_by_gpt(comment)
    print(f"댓글: {comment}\n감정: {sentiment}\n")
    time.sleep(0.06)  # API 속도 조절을 위한 지연 시간 설정

댓글: 이번 신곡 너무 좋아
감정: 2

댓글: 국힙원탑 희진누님의 위대한 작품
감정: 2

댓글: 팜하니 너무 귀여워서ㅠㅠㅠ‍️🫶
감정: 2



In [6]:
from tqdm import tqdm
tqdm.pandas()

df['sentiment_by_gpt'] = df['comment'].progress_apply(sentiment_by_gpt)

100%|██████████| 500/500 [05:20<00:00,  1.56it/s]


In [7]:
import google.generativeai as genai
import os

key_path = '/Users/jaesolshin/key/gemini_key.txt'
apikey = open(key_path, 'r', encoding='utf-8').read()
genai.configure(api_key=apikey)

system_instruction = "문장의 감정을 '2' (긍정), '1' (중립), '0' (부정) 중 하나로 평가해 주세요. {<텍스트 형식의 숫자>}"
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=system_instruction) # 모델 정의할 때 system_instruction 부여

# Sample 텍스트(50~52번 댓글)
comments = df['comment'][50:53]  # 슬라이싱 처리

def sentiment_by_gemini(comment):
    response = model.generate_content(comment)

    #print(response)  # 또는 response의 타입과 내용을 명확히 보기 위한 출력
    if not response.candidates:
        return "N/A"
    
    if not response.candidates[0].content.parts:
        return "N/A"
    
    response_text = response.candidates[0].content.parts[0].text.strip()
    
    score = re.search(r'\d', response_text)
    return score.group() if score else "N/A"

# 감정 분석 수행 및 결과 출력
for comment in comments:
    sentiment = sentiment_by_gemini(comment)
    print(f"댓글: {comment}\n감정: {sentiment}\n")

댓글: 이번 신곡 너무 좋아
감정: 2

댓글: 국힙원탑 희진누님의 위대한 작품
감정: 2

댓글: 팜하니 너무 귀여워서ㅠㅠㅠ‍️🫶
감정: 2



In [8]:
df['sentiment_by_gemini'] = df['comment'].progress_apply(sentiment_by_gemini)

100%|██████████| 500/500 [03:24<00:00,  2.44it/s]


In [29]:
ternary_df_llm.dtypes

ternary_js               int64
sentiment_by_gpt         int64
sentiment_by_gemini    float64
dtype: object

In [60]:
ternary_df_llm = df[['ternary_km', 'sentiment_by_gpt', 'sentiment_by_gemini']]
ternary_df_llm = ternary_df_llm[ternary_df_llm['sentiment_by_gemini']!='N/A']
ternary_df_llm.dropna()

ternary_df_llm['sentiment_by_gpt'] = pd.to_numeric(ternary_df_llm['sentiment_by_gpt'], errors='coerce').astype(int)
ternary_df_llm['sentiment_by_gemini'] = pd.to_numeric(ternary_df_llm['sentiment_by_gemini'], errors='coerce').astype(int)

print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df_llm)}")
#print(f"삼진분류 Fleiss' Kappa: {fleiss_kappa(ternary_df_llm)}")

삼진분류 Krippendorff's Alpha: 0.6429655858768734


In [48]:
ternary_df_llm = df[['sentiment_by_gpt', 'sentiment_by_gemini']]
print(f"삼진분류 Krippendorff's Alpha: {krippendorffs_alpha(ternary_df_llm)}")

삼진분류 Krippendorff's Alpha: 0.6691845739396597
